In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path('../backend').resolve()))

from src.image_generator import PCAStyleGan
import pandas as pd
import numpy as np

image_generator = PCAStyleGan()

tasks = pd.read_parquet('../data/integrated/GANSLIDER_TASKS_TRANSFORMED.parquet')
tasks = tasks[~tasks.edit_value.isna()]

conf = pd.read_parquet('../data/integrated/GANSLIDER_CONFIGS_FLAT.parquet')
conf = conf[['taskid','slider_id', 'target_value', 'seed']].drop_duplicates()

merged = tasks.merge(conf, on=['taskid', 'slider_id'])
merged

In [ ]:
export = merged[['taskid', 'slider_type', 'num_sliders', 'completed', 'seed']].drop_duplicates()
export['file_name'] = export['slider_type'] + '_' +  export['num_sliders'].apply(lambda v: f'{v:02d}') + '_' + export['completed'] + '_' + export['taskid']  + '.png'
export

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

outdir = '../images/user_generated_images'

for idx, row in tqdm(export.iterrows()):
	sample_task = merged.loc[merged.taskid == row['taskid'], ['slider_id', 'edit_value', 'target_value', 'seed']]
	seed = row['seed']
	
	user_edits = sample_task[['slider_id', 'edit_value']].rename(columns={'slider_id': 'id', 'edit_value': 'value'})
	user_edits = user_edits.to_dict(orient='records')

	target_edits = sample_task[['slider_id', 'target_value']].rename(columns={'slider_id': 'id', 'target_value': 'value'})
	target_edits = target_edits.to_dict(orient='records')

	user_img = image_generator.generate_image(edits=user_edits, seed=seed, size=256)
	targ_img = image_generator.generate_image(edits=target_edits, seed=seed, size=256)

	fig, axes = plt.subplots(ncols=2)
	axes[0].imshow(user_img)
	axes[1].imshow(targ_img)
	plt.savefig(Path(outdir) / row['file_name'])
	plt.close()

In [7]:
export = merged[['taskid', 'slider_type', 'num_sliders', 'completed', 'seed']].drop_duplicates()
export['user_img'] = export['taskid'] + '_user.png'
export['targ_img'] = export['taskid'] + '_targ.png'
export

,taskid,slider_type,num_sliders,completed,seed,user_img,targ_img
0,f50de36a-f183-4527-af74-f55ac6f69b13,REGULAR,8,DONE,120,f50de36a-f183-4527-af74-f55ac6f69b13_user.png,f50de36a-f183-4527-af74-f55ac6f69b13_targ.png
8,bf7bc7ad-5b70-48f0-89f0-0d935c075ac7,REGULAR,2,DONE,1178,bf7bc7ad-5b70-48f0-89f0-0d935c075ac7_user.png,bf7bc7ad-5b70-48f0-89f0-0d935c075ac7_targ.png
10,f33744ee-91d4-4175-aa1e-e50bd950f063,REGULAR,3,DONE,7735,f33744ee-91d4-4175-aa1e-e50bd950f063_user.png,f33744ee-91d4-4175-aa1e-e50bd950f063_targ.png
13,7b578edf-7230-4c6c-9f88-dc452e389d56,FILMSTRIP,2,DONE,3146,7b578edf-7230-4c6c-9f88-dc452e389d56_user.png,7b578edf-7230-4c6c-9f88-dc452e389d56_targ.png
15,a520f54b-c5fe-4fd9-ac00-c7dd04c6993f,REGULAR,4,DONE,5526,a520f54b-c5fe-4fd9-ac00-c7dd04c6993f_user.png,a520f54b-c5fe-4fd9-ac00-c7dd04c6993f_targ.png
...,...,...,...,...,...,...,...
9089,f8dcd5e1-2746-497b-bee6-9bfb430db283,REGULAR,3,DONE,7960,f8dcd5e1-2746-497b-bee6-9bfb430db283_user.png,f8dcd5e1-2746-497b-bee6-9bfb430db283_targ.png
9092,65e6f397-2ae3-4071-8a40-7ac78c00df7b,REGULAR,10,DONE,7631,65e6f397-2ae3-4071-8a40-7ac78c00df7b_user.png,65e6f397-2ae3-4071-8a40-7ac78c00df7b_targ.png
9102,1fc44ae0-0230-4b50-b6ef-906c09adf6d8,FILMSTRIP,8,DONE,5735,1fc44ae0-0230-4b50-b6ef-906c09adf6d8_user.png,1fc44ae0-0230-4b50-b6ef-906c09adf6d8_targ.png
9110,5bf02d4f-49da-4d5a-8d04-2811e4bad8d8,REGULAR,2,DONE,9790,5bf02d4f-49da-4d5a-8d04-2811e4bad8d8_user.png,5bf02d4f-49da-4d5a-8d04-2811e4bad8d8_targ.png


In [28]:
export.to_csv(f'../data/user_generated_images_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.csv')

In [9]:
import matplotlib.pyplot as plt
from tqdm import tqdm

outdir = '../images/user_generated_images'

for idx, row in tqdm(export.iterrows(), total=export.shape[0]):
	sample_task = merged.loc[merged.taskid == row['taskid'], ['slider_id', 'edit_value', 'target_value', 'seed']]
	seed = row['seed']
	
	user_edits = sample_task[['slider_id', 'edit_value']].rename(columns={'slider_id': 'id', 'edit_value': 'value'})
	user_edits = user_edits.to_dict(orient='records')

	target_edits = sample_task[['slider_id', 'target_value']].rename(columns={'slider_id': 'id', 'target_value': 'value'})
	target_edits = target_edits.to_dict(orient='records')

	user_img = image_generator.generate_image(edits=user_edits, seed=seed, size=256)
	targ_img = image_generator.generate_image(edits=target_edits, seed=seed, size=256)

	user_img.save('../images/user_generated_images_single/' + row['user_img'], "PNG")
	targ_img.save('../images/user_generated_images_single/' + row['targ_img'], "PNG")

100%|██████████| 1932/1932 [07:28<00:00,  4.31it/s]
